Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give read access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [1]:
## DONT CHANGE THIS CELL
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip

--2021-10-24 18:08:07--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip [following]
--2021-10-24 18:08:08--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217313 (212K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 212.22K   113KB/s    in 1.9s    

2021-10-24 18:08:11 (113 KB/s) - ‘data.zip’ saved [217313/217313]



In [2]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 43.5 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 3.3 MB 34.7 MB/s 
     |████████████████████████████████| 895 kB 47.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install datasets

     |████████████████████████████████| 290 kB 5.6 MB/s 
     |████████████████████████████████| 243 kB 40.0 MB/s 
     |████████████████████████████████| 125 kB 44.2 MB/s 
     |████████████████████████████████| 1.3 MB 33.1 MB/s 
     |████████████████████████████████| 271 kB 49.8 MB/s 
     |████████████████████████████████| 160 kB 46.5 MB/s 


In [16]:
## Import relevant packages
import pandas as pd
import numpy as np
import csv
import torch
import re
from datasets import load_metric,load_dataset
from transformers import Trainer
from transformers import TrainingArguments
from transformers import BertModel, BertConfig,BertForSequenceClassification,BertForMaskedLM,BertLMHeadModel,DistilBertModel,AutoModelForSequenceClassification
from transformers import pipeline,DataCollatorWithPadding
from transformers import AutoTokenizer,AutoTokenizer,BertTokenizerFast,BertTokenizer,DistilBertTokenizer,DistilBertTokenizerFast,DistilBertForMaskedLM,DistilBertForSequenceClassification
import datasets

In [22]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased',padding=True,truncation=True)

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a

In [23]:
data_collator=DataCollatorWithPadding(tokenizer)

In [7]:
!unzip 'data.zip'

Archive:  data.zip
   creating: data/
   creating: data/train/
  inflating: data/train/train.gold.txt  
  inflating: data/train/train.data.txt  
   creating: data/validation/
  inflating: data/validation/validation.data.txt  
  inflating: data/validation/validation.gold.txt  


In [12]:
def preprocess(sen):
    sen=re.sub("\W"," ",sen)
    sen=re.sub("\s+"," ",sen)
    return sen.lower().strip()

In [24]:
dfvx = pd.read_csv('./data/validation/validation.data.txt', delimiter = "\t",header=None)
dfvy = pd.read_csv('./data/validation/validation.gold.txt', delimiter = "\t",header=None)
dfvy=dfvy.applymap(lambda x: x=='F')
dfvy[0]=dfvy[0].astype(int)
dfvx['label']=dfvy[0]
dfvx['text1']=dfvx[3]
dfvx['text2']=dfvx[4]
dfvx=dfvx[['text1','text2','label']]
dfvx['text1']=dfvx['text1'].map(lambda x:preprocess(x))
dfvx['text2']=dfvx['text2'].map(lambda x:preprocess(x))
dfvx=np.array(dfvx)
with open('./formattedval.csv','w') as f:
  writer =csv.writer(f)
  writer.writerow(['text1','text2','label'])
  writer.writerows(dfvx)
  npvx=load_dataset('csv', data_files='./formattedval.csv')

Using custom data configuration default-39e688d519ba14f8


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-39e688d519ba14f8/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
dfx = pd.read_csv('./data/train/train.data.txt', delimiter = "\t",header=None)
dfy = pd.read_csv('./data/train/train.gold.txt', delimiter = "\t",header=None)
dfy=dfy.applymap(lambda x: x=='F')
dfy[0]=dfy[0].astype(int)
dfx['label']=dfy[0]
dfx['text1']=dfx[3]
dfx['text2']=dfx[4]
dfx=dfx[['text1','text2','label']]
dfx['text1']=dfx['text1'].map(lambda x:preprocess(x))
dfx['text2']=dfx['text2'].map(lambda x:preprocess(x))
dfx=np.array(dfx)
with open('./formatted.csv','w') as f:
  writer =csv.writer(f)
  writer.writerow(['text1','text2','label'])
  writer.writerows(dfx)
npx=load_dataset('csv', data_files='./formatted.csv')

Using custom data configuration default-e2268ac8b260859c


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e2268ac8b260859c/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache 

In [29]:
def tokenize_func(row):
  return tokenizer(row['text1'],row['text2'],padding=True,truncation=True)
tokenized_data=npx.map(tokenize_func,batched=True)
tokenized_val_data=npvx.map(tokenize_func,batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [30]:
tokenized_data = tokenized_data.map(lambda example: {}, remove_columns=['text1','text2'])
tokenized_val_data = tokenized_val_data.map(lambda example: {}, remove_columns=['text1','text2'])

  0%|          | 0/5428 [00:00<?, ?ex/s]

  0%|          | 0/584 [00:00<?, ?ex/s]

In [31]:
training_args = TrainingArguments("test_trainer",evaluation_strategy="epoch")
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_data['train'],eval_dataset=tokenized_val_data['train'],tokenizer=tokenizer,data_collator=data_collator,compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [32]:
trainer.train()

***** Running training *****
  Num examples = 5428
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2037


Epoch,Training Loss,Validation Loss,Accuracy
1,0.603000,0.642309,0.703767
2,0.399800,1.084177,0.691781
3,0.141300,1.556444,0.695205


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 584
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 584
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/che

TrainOutput(global_step=2037, training_loss=0.3552689292469502, metrics={'train_runtime': 557.8009, 'train_samples_per_second': 29.193, 'train_steps_per_second': 3.652, 'total_flos': 465531845802000.0, 'train_loss': 0.3552689292469502, 'epoch': 3.0})

In [33]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 584
  Batch size = 8


{'epoch': 3.0,
 'eval_accuracy': 0.6952054794520548,
 'eval_loss': 1.5564439296722412,
 'eval_runtime': 5.6815,
 'eval_samples_per_second': 102.79,
 'eval_steps_per_second': 12.849}

In [35]:
trainer.save_model('./my-bert-prerained')

Saving model checkpoint to ./my-bert-prerained
Configuration saved in ./my-bert-prerained/config.json
Model weights saved in ./my-bert-prerained/pytorch_model.bin
tokenizer config file saved in ./my-bert-prerained/tokenizer_config.json
Special tokens file saved in ./my-bert-prerained/special_tokens_map.json
